In [ ]:
#! pip uninstall torch torchvision torchaudio --yes

Found existing installation: torch 2.6.0+cu118
Uninstalling torch-2.6.0+cu118:
  Successfully uninstalled torch-2.6.0+cu118
Found existing installation: torchvision 0.21.0+cu118
Uninstalling torchvision-0.21.0+cu118:
  Successfully uninstalled torchvision-0.21.0+cu118
Found existing installation: torchaudio 2.6.0+cu118
Uninstalling torchaudio-2.6.0+cu118:
  Successfully uninstalled torchaudio-2.6.0+cu118


In [2]:
#! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
   ---------------------------------------- 0.0/2.9 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.9 GB 4.8 MB/s eta 0:10:05
   ---------------------------------------- 0.0/2.9 GB 5.8 MB/s eta 0:08:18
   ---------------------------------------- 0.0/2.9 GB 5.6 MB/s eta 0:08:39
   ---------------------------------------- 0.0/2.9 GB 5.8 MB/s eta 0:08:19
   ---------------------------------------- 0.0/2.9 GB 5.8 MB/s eta 0:08:23
   ---------------------------------------- 0.0/2.9 GB 6.0 MB/s eta 0:08:01
   ---------------------------------------- 0.0/2.9 GB 6.0 MB/s eta 0:08:01
   ---------------------------------------- 0.0/2.9 GB 6.1 MB/s eta 0:07:52
   ---------------------------------------- 0.0/2.9 GB 6.2 MB/s eta 0:07:49
   ---------------------------------------- 0.0/2.9 GB 6.2 MB/s eta 0:07:47
   ---------------------------------------- 0.0/2.9 GB 6.1 MB/s eta 0:07:51
   ---------------------------------

In [3]:
#! pip install ultralytics


In [ ]:
# import required packages
import cv2
import numpy as np
import os
'pip install ultralytics'
from ultralytics import YOLO
import math 
import sys
current_dir = os.getcwd()
parent_dir = os.path.abspath("../../")
sys.path.append(parent_dir)
from Update_Git import git_add, git_commit, git_push
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

### GitHub

In [ ]:
path = os.path.join(current_dir, "yolo_1.ipynb")
for i in range(26):
    if os.path.exists(path):
        git_add(path)
        git_commit("Add yolo_1.ipynb")
        git_push("main")
    else:
        print("File does not exist.")

In [ ]:
pip install ultralytics --upgrade


### Model

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')  # this should auto-download if missing

classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

### Apply YOLO to an image

In [ ]:
# Load an image
img_path = "data/test/dog.jpg"
image = cv2.imread(img_path)

# Apply the model to the image
results = model(img_path)

# Show the results
results[0].show()
results[0].save(os.path.join(current_dir, r"data\results\dog_output.jpg"))


In [ ]:
# Draw detections manually

# Loop through each detection result (e.g., one per image)
for result in results:
    
    # Loop through each detected bounding box in the result
    for box in result.boxes:
        
        # Extract the coordinates of the bounding box corners: top-left (x1, y1) and bottom-right (x2, y2)
        # `box.xyxy[0]` is a tensor like [x1, y1, x2, y2], so we convert it to a list
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        
        # Get the predicted class index (e.g., 0 = dog, 1 = cat, etc.)
        cls = int(box.cls[0])
        
        # Get the confidence score (probability that the detection is correct)
        conf = box.conf[0]
        
        # Build a label string like "dog 0.92"
        label = f"{classNames[cls]} {conf:.2f}"
        
        # Draw the bounding box on the image (blue rectangle)
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
        
        # Draw the class label and confidence above the bounding box
        cv2.putText(image, label, (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

# Display the image
cv2.imshow("YOLOv8 Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


### Apply YOLO to webcam

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

while True:
    success, img = cap.read()
    results = model(img, stream=True) #It returns a generator that yields results one at a time — rather than returning a full list all at once.

    # coordinates
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

            # put box in cam
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 3)

            # confidence
            confidence = math.ceil((box.conf[0]*100))/100
            print("Confidence --->",confidence)

            # class name
            cls = int(box.cls[0])
            print("Class name -->", classNames[cls])

            # object details
            org = [x1, y1]
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = (0, 0, 255)
            thickness = 2

            cv2.putText(img, classNames[cls], org, font, fontScale, color, thickness)

    cv2.imshow('Webcam', img)
    if cv2.waitKey(200) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()